In [12]:
from time import sleep
import cyschoolhousesuite as cysh
from pandas import read_csv

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [45]:
def search_for_section(driver, sec_num):
    driver.find_element_by_class_name("searchTextBox").send_keys(sec_num)
    driver.find_element_by_name("search").click()
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "section_intervention_data")))
    driver.find_element_by_name("section_intervention_data").click()

def enrollment_window(driver):
    driver.switch_to_window('4294967331')
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "addButton")))
    driver.find_element_by_id("addButton").click()
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "j_id0:j_id48:enrollDate")))
    driver.find_element_by_id("j_id0:j_id48:enrollDate").clear()
    driver.find_element_by_id("j_id0:j_id48:enrollDate").send_keys('8/14/2017')
    
def enter_students_into_enrollment(driver, sfid):
    table = driver.find_element_by_xpath("//div[@class='table-box']/table")
    student = table.find_element_by_xpath("//tr/td[contains(text(), " + sfid + ")]")
    student.click() #element returned unclickable
    return None

def input_data():
    data = read_csv('input_files/section-key.csv')
    return data.to_dict(orient='records')

def test_script():
    inputs = input_data()[0]
    driver = cysh.get_driver()
    cysh.open_cyschoolhouse18_sb(driver)
    search_for_section(driver, 74013)
    enrollment_window(driver)
    enter_students_into_enrollment(driver, inputs['Student ID'])
    #sleep(5)
    #driver.quit()

In [46]:
test_script()

ElementNotInteractableException: Message: 
